In [ ]:
# Blue Coat log analysis notebook
=================================================

## Loading the logs as tables from parquet files in HDFS

The following code loads the access logs from the BlueCoat/accesslog directory in the user's HDFS directory and makes them available as a table.

'data' is a dataframe that contains rows selected by the SQL query.

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

if __name__ == '__main__':
    sqlContext = SQLContext(sc)

    df = sqlContext.load('BlueCoat/accessLog')
    sqlContext.registerDataFrameAsTable(df, "accesslog")
    df2 = sqlContext.load('reputation/alienvault')
    sqlContext.registerDataFrameAsTable(df2, 'av')
    data = sqlContext.sql("SELECT clientip, username, host, categories, count(*) as hits FROM accesslog JOIN av WHERE host=ip GROUP BY clientip, username, host, categories ORDER BY hits DESC") 
    data.cache()
    data.show()

## Plotting a pie chart from the data
The following code uses the 'matplotlib' to draw a pie chart from 'data'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

clientip = []
hits = []
for i in data.take(5):
    hits.append(i.hits)
    clientip.append(i.clientip)

plt.pie(hits, labels=clientip)

## Plotting a bar graph from the data
The following code uses the 'matplotlib' to draw a pie chart from 'data'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline  

clientip = []
hits = []
for i in data.take(5):
    hits.append(i.hits)
    clientip.append(i.clientip)
    
N = 5
ind = np.arange(N)  # the x locations for the groups
width = 0.5     # the width of the bars
fig = plt.figure()
ax = fig.add_subplot(111)
rects1 = ax.bar(ind, hits, width, color='b')

# add some
ax.set_ylabel('hits')
ax.set_title('Top 5 denied clients')
ax.set_xticks(ind+width)
ax.set_xticklabels(clientip)

In [ ]:
!ls